## Imports

In [2]:
import sys
import random
from pathlib import Path
from collections import Counter

import datasets
from datasets import load_dataset_builder, load_dataset

from utils import replace_text_with_whitespace

In [3]:
ELK_PATH = Path("../../../elk/")
ELK_PATH.resolve()

PosixPath('/rds/user/am3052/hpc-work/elk')

In [4]:
modules = [
    ELK_PATH,
    ELK_PATH / "elk" / "promptsource",
]

for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

sys.path[:2]

['/rds/user/am3052/hpc-work/elk/elk/promptsource',
 '/rds/user/am3052/hpc-work/elk']

In [5]:
from templates import DatasetTemplates

In [6]:
# Disable the logging of the datasets library
import datasets

datasets.logging.set_verbosity_error()

## Config

In [6]:
# all datasets used in Burns et al. (2022) (apart from story_cloze)
BURNS_DATASETS = [
    "ag_news",
    "amazon_polarity",
    "dbpedia_14",
    "glue:qnli",
    "imdb",
    "piqa",
    "super_glue:boolq",
    "super_glue:copa",
    "super_glue:rte",
]

SPLIT = "validation"

# These numbers are chosen so that both datasets have
# approximately 10k examples in total
N_PER_DATASET = 1200 if SPLIT == "train" else 1800
if SPLIT == "train":
    assert N_PER_DATASET <= 2490, "N_PER_DATASET must be <= 2490 for train"
else:
    assert N_PER_DATASET <= 1838, "N_PER_DATASET must be <= 1838 for validation"

## Load and inspect the datasets

In [7]:
dataset_dict = {}
for dataset_path in BURNS_DATASETS:
    print(dataset_path)

    # Parse dataset name
    dataset_name = None    
    if ":" in dataset_path:
        dataset_path, dataset_name = dataset_path.split(":")
    
    
    # Get the most validation-like split
    available_splits = load_dataset_builder(
        dataset_path, name=dataset_name
    ).info.splits.keys()
    split = "validation" if "validation" in available_splits else "test"
    split = split if SPLIT != "train" else "train"
    print(f"{split=}")

    # Load the dataset
    dataset = load_dataset(
        dataset_path, name=dataset_name, split=split,
    )

    # Get a desired subset of the data
    n = N_PER_DATASET if dataset.num_rows > N_PER_DATASET else dataset.num_rows
    dataset = dataset.shuffle(seed=42).select(range(n))

    print(dataset.num_rows)

    key = f"{dataset_path}/{dataset_name}" if dataset_name else dataset_path
    dataset_dict[key] = dataset

    print("-----------------------------------")

ag_news
split='test'
1800
-----------------------------------
amazon_polarity
split='test'
1800
-----------------------------------
dbpedia_14
split='test'
1800
-----------------------------------
glue:qnli
split='validation'
1800
-----------------------------------
imdb
split='test'
1800
-----------------------------------
piqa
split='validation'
1800
-----------------------------------
super_glue:boolq
split='validation'
1800
-----------------------------------
super_glue:copa
split='validation'
100
-----------------------------------
super_glue:rte
split='validation'
277
-----------------------------------


In [8]:
assert all([len(Counter(dataset["label"])) > 1 for dataset in dataset_dict.values()])

In [9]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {dataset.num_rows}")

ag_news: 1800
amazon_polarity: 1800
dbpedia_14: 1800
glue/qnli: 1800
imdb: 1800
piqa: 1800
super_glue/boolq: 1800
super_glue/copa: 100
super_glue/rte: 277


In [10]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

ag_news: 4
amazon_polarity: 2
dbpedia_14: 14
glue/qnli: 2
imdb: 2
piqa: 2
super_glue/boolq: 2
super_glue/copa: 2
super_glue/rte: 2


## Split into validation and test

In [11]:
ppo_dataset_dict = {}
test_dataset_dict = {}

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)
    splits = dataset.train_test_split(test_size=0.22, seed=42)
    # print(splits)
    print(splits["train"].num_rows, splits["test"].num_rows)

    ppo_dataset_dict[dataset_name] = splits["train"]
    test_dataset_dict[dataset_name] = splits["test"]
    print("-----------------------------------")

ag_news
1404 396
-----------------------------------
amazon_polarity
1404 396
-----------------------------------
dbpedia_14
1404 396
-----------------------------------
glue/qnli
1404 396
-----------------------------------
imdb
1404 396
-----------------------------------
piqa
1404 396
-----------------------------------
super_glue/boolq
1404 396
-----------------------------------
super_glue/copa
78 22
-----------------------------------
super_glue/rte
216 61
-----------------------------------


In [12]:
sum(dataset.num_rows for dataset in ppo_dataset_dict.values()), sum(dataset.num_rows for dataset in test_dataset_dict.values())

(10122, 2855)

In [17]:
# datasets.DatasetDict(test_dataset_dict).save_to_disk("datasets/test_dataset_raw")

## Get the templates

In [7]:
# Set dataset_dict to test_dataset
dataset_dict = datasets.DatasetDict.load_from_disk("datasets/test_dataset_raw")
dataset_dict

DatasetDict({
    ag_news: Dataset({
        features: ['text', 'label'],
        num_rows: 396
    })
    amazon_polarity: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 396
    })
    dbpedia_14: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 396
    })
    glue/qnli: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 396
    })
    imdb: Dataset({
        features: ['text', 'label'],
        num_rows: 396
    })
    piqa: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 396
    })
    super_glue/boolq: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 396
    })
    super_glue/copa: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 22
    })
    super_glue/rte: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 61
    })
})

In [8]:
sum(dataset.num_rows for dataset in dataset_dict.values())

2855

In [9]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

ag_news: 4
amazon_polarity: 2
dbpedia_14: 14
glue/qnli: 2
imdb: 2
piqa: 2
super_glue/boolq: 2
super_glue/copa: 2
super_glue/rte: 2


In [10]:
dataset_template_dict = {}

for dataset_path in dataset_dict.keys():
    dataset_templates = DatasetTemplates(dataset_path)

    dataset_templates.templates = {
        x.name: x for x in dataset_templates.templates.values() if x.get_answer_choices_list(dataset_dict[dataset_path][0]) is not None
    }

    dataset_template_dict[dataset_path] = dataset_templates

In [11]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 15
amazon_polarity: 11
dbpedia_14: 11
glue/qnli: 5
imdb: 13
piqa: 7
super_glue/boolq: 10
super_glue/copa: 9
super_glue/rte: 11


## Inspect the prompts

In [12]:
# dataset_name = "ag_news"

# for template_name, template in dataset_template_dict[dataset_name].templates.items():
#     # print(template_name)
#     # print(dataset_dict[dataset_name][0])
#     q, a = template.apply(
#         dataset_dict[dataset_name][0]
#     )
#     # print(q == q.strip())
#     # print(a == a.strip())
#     # print(" ".join([q, a.strip()]))
#     print(" ".join([q, a]))
#     print("---------------------------------")

## Form the dataset for the chosen split

In [13]:
# for dataset_name, dataset_templates in dataset_template_dict.items():
#     print(dataset_name)
#     for template_name, template in dataset_templates.templates.items():
#         print(f"{template_name}: {template.get_fixed_answer_choices_list()}")

#     print("---------------------------------")


In [14]:
# for dataset_name, dataset_templates in dataset_template_dict.items():
#     print(dataset_name)
#     for template_name, template in dataset_templates.templates.items():
#         print(f"{template_name}: {template.get_answer_choices_list(dataset_dict[dataset_name][0])}")

#     print("---------------------------------")

In [15]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 15
amazon_polarity: 11
dbpedia_14: 11
glue/qnli: 5
imdb: 13
piqa: 7
super_glue/boolq: 10
super_glue/copa: 9
super_glue/rte: 11


In [16]:
%%time

# Reproducibility
random.seed(2023)

ALLOWED_KEYS = ["text", "label", "original_dataset", "template_name"]

new_dataset = []

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)

    # if dataset_name != "ag_news": continue
    # if dataset_name != "imdb": continue
    # if dataset_name != "piqa": continue

    for idx, entry in enumerate(dataset):
        new_entry = entry.copy()
        
        # In case we need to know which dataset the entry came from
        new_entry["original_dataset"] = dataset_name

        # Sample a random template
        template_name = random.choice(
            list(dataset_template_dict[dataset_name].templates.keys())
        )
        template = dataset_template_dict[dataset_name].templates[template_name]
        new_entry["template_name"] = template_name

        # Whether the sample will be truthful or not
        is_truthful = random.choice([True, False])

        # If not truthful, sample a random incorrect label
        if not is_truthful and len(dataset.features["label"].names) > 2:
            label_mappping = dataset.features["label"] # [class 1, class 2, ...]
            all_label_ids = [label_mappping.str2int(x) for x in label_mappping.names] # [0, 1, ...]
            incorrect_label_id = all_label_ids[(1 - new_entry["label"]) % len(all_label_ids)]
            incorrect_label = template.get_fixed_answer_choices_list()[incorrect_label_id]
            
        
        # Apply the template
        if is_truthful:
            new_text = " ".join(template.apply(new_entry))
        elif not is_truthful and len(dataset.features["label"].names) > 2:
            q, a = template.apply(new_entry)
            incorrect_label = replace_text_with_whitespace(a, incorrect_label)
            new_text = " ".join([q, incorrect_label])
        else:
            # Untruthful binary case
            new_entry["label"] = 1 - new_entry["label"]
            new_text = " ".join(template.apply(new_entry))

        new_entry["text"] = new_text

        # We can now change the label to whether the sample is truthful or not
        new_entry["label"] = int(is_truthful)

        # Remove all other keys
        new_entry = {k: v for k, v in new_entry.items() if k in ALLOWED_KEYS}

        # Append to the new dataset
        new_dataset.append(new_entry)

ag_news
amazon_polarity
dbpedia_14
glue/qnli
imdb
piqa
super_glue/boolq
super_glue/copa
super_glue/rte
CPU times: user 6.09 s, sys: 15.6 ms, total: 6.11 s
Wall time: 6.24 s


In [17]:
my_dataset = datasets.Dataset.from_list(new_dataset)
my_dataset

Dataset({
    features: ['text', 'label', 'original_dataset', 'template_name'],
    num_rows: 2855
})

In [18]:
my_dataset[0]

{'text': "Consider the following example: ''' Chicken Genome Sheds New Light on Human DNA A new study states that 60 of the genes in chicken have close relations to human DNA. This may not comfort those who frequently eat the creature, but may ponder this the next time they order a batch of chicken wings. ''' Choice 1: Science and technology. Choice 2: Business.Between choice 1 and choice 2, what is the topic of this example? Business",
 'label': 0,
 'original_dataset': 'ag_news',
 'template_name': 'burns_2'}

In [19]:
Counter(my_dataset["label"]), Counter(my_dataset["original_dataset"])

(Counter({0: 1476, 1: 1379}),
 Counter({'ag_news': 396,
          'amazon_polarity': 396,
          'dbpedia_14': 396,
          'glue/qnli': 396,
          'imdb': 396,
          'piqa': 396,
          'super_glue/boolq': 396,
          'super_glue/copa': 22,
          'super_glue/rte': 61}))

In [20]:
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_validation.parquet")

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1671266

In [21]:
!ls -lah datasets

total 5.8M
drwxrwsr-x 4 am3052 am3052 4.0K Jun 14 23:54 .
drwxrwsr-x 3 am3052 am3052 4.0K Jun 14 22:26 ..
-rw-rw-r-- 1 am3052 am3052 2.8M Jun 14 16:08 burns_datasets_VINC_train.parquet
-rw-rw-r-- 1 am3052 am3052 858K Jun 14 23:54 burns_datasets_VINC_validation.parquet
drwxrwsr-x 9 am3052 am3052 4.0K Jun 14 22:18 ppo_dataset_raw
-rw-rw-r-- 1 am3052 am3052 3.0M Jun 14 23:53 ppo_training_dataset.parquet
drwxrwsr-x 9 am3052 am3052 4.0K Jun 14 22:18 test_dataset_raw


In [22]:
parquet_data_files = {
    "train": "datasets/my_dataset_train.parquet",
    "validation": "datasets/my_dataset_validation.parquet",
}

my_dataset = load_dataset("parquet", data_files=parquet_data_files)
my_dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /home/am3052/.cache/huggingface/datasets/parquet/default-73efdde8c9f753c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'original_dataset', 'template_name'],
        num_rows: 900
    })
    validation: Dataset({
        features: ['text', 'label', 'original_dataset', 'template_name'],
        num_rows: 900
    })
})